# Notebook to edit data

In [1]:
import pygsheets
import pandas as pd
import numpy as np
import datetime
import util
c = pygsheets.authorize(outh_file='other.json')



# Protected Range

In [ ]:
gc = pygsheets.authorize(outh_file='/home/denis/Program/work/sheets.googleapis.com-python.json')
print(gc.get_range('1B1vEqp6uK7x2B8pV0dSH3S9DetFesaDP8n83Ewy53Yk', 'A1:D3' ))
gridrange = {
  "sheetId": 0,
  "startRowIndex": 0,
  "endRowIndex": 2,
  "startColumnIndex": 0,
  "endColumnIndex": 2,
}

editors = {
  "users": [
    'ffbskt@gmail.com',
  ],
  "groups": [
  ],
  "domainUsersCanEdit": True,
}
request = {"addProtectedRange": {
            "protectedRange": {
                "range": gridrange,
                "editors": editors
            },

        }}
#print(gc.sh_batch_update('1B1vEqp6uK7x2B8pV0dSH3S9DetFesaDP8n83Ewy53Yk', request, None, False))

In [2]:
gc = pygsheets.authorize(outh_file='/home/denis/Program/work/sheets.googleapis.com-python.json')
wks = util.read_sheet('1s5vtpVMmWZQCuUOdBVoNCCAVK_7U6ax3uZn0ynzgB28')
df = wks.get_as_df()

In [22]:
def open_protected_today(key):
    col = datetime.datetime.now().strftime("%m-%d")
    ind = df.columns.get_loc(col)
    
    #delete protected range
    request = {
              "deleteProtectedRange": {
                "protectedRangeId": 1,
              }
            }
    try:
        gc.sh_batch_update(key, request, None, False)
    except:
        pass
    #set new range
    gridrange = {
      "sheetId": 0,
      "startRowIndex": 0,
      "endRowIndex": 50,
      "startColumnIndex": 0,
      "endColumnIndex": ind,
    }
    editors = {
      "users": [
        'ffbskt@gmail.com',
      ],
      "groups": [
      ],
      "domainUsersCanEdit": True,
    }
    request = {"addProtectedRange": {
                "protectedRange": {
                "protectedRangeId": 1,
                "range": gridrange,
                "editors": editors
            },

        }}
    gc.sh_batch_update(key, request, None, False)
    ##open col of protection (can edit by link)

#create protected range for today.    
    
open_protected_today('1s5vtpVMmWZQCuUOdBVoNCCAVK_7U6ax3uZn0ynzgB28') 

KeyError: '10-10'

In [5]:
def first_empty(df, col):
    r = df.loc[[col]] != ''
    for ind, val in enumerate(r.values[0]):
        if not val:
            return ind
#first_empty()

# Copy sheet and add to graph

In [13]:
wks = util.read_sheet('1zx2VAIuiqFP9lZxwMUxeEyih7qpa2p5Su4mOw_ilC4E')#.get_as_df()
c.create(title='Quise', parent_id='1ncBfOFctKPVRKBYwrc58U5_UaD-d0D30')
wks_copy = util.read_sheet(c.spreadsheetId)
wks_copy.set_dataframe(wks.get_as_df(), start='A1')

# Edit data in sheet

In [14]:
df = wks_copy.get_as_df()
df = util.join_suname_to_init(df, crange=[2,5], rrange=None, remove=True)

In [15]:
df = df.rename(index=str, columns={"Фамилия": "ФИО", 
                              "Адрес электронной почты": "Почта",
                             "Учебная группа": "Группа",
                             "Секция по физкультуре": "Секция",
                             "Медецинская группа, как на фотографии": "справка"
                             })
df

,Отметка времени,Почта,ФИО,Имя,Отчество,Фотография лица,Еще одна фотография (желательно изменить угол на 5 - 10* и освещение),Еще одна фотография (желательно изменить угол на 5 - 10* и освещение),Фотография справки о медосмотре с указанием группы здоровья.,Группа,...,,,,,,,,,,
0,18.03.2018 10:55:33,ffbskt@gmail.com,Волконский Д А,Денис,Алексеевич,https://drive.google.com/open?id=1VzOdUQLBJo6X...,https://drive.google.com/open?id=1Jw7AgW79xJpL...,https://drive.google.com/open?id=1Q3Uw5u_qJyse...,,0,...,,,,,,,,,,
1,18.03.2018 11:10:08,e_poved@mail.ru,Поведская Е Б,Екатерина,Борисовна,,,,,Основная,...,,,,,,,,,,
2,18.03.2018 13:34:36,musin.dd17@physic.msu.ru,Мусин Д Д,Даниил,Денисович,https://drive.google.com/open?id=1Fpr6eUEvtYY9...,,,,107,...,,,,,,,,,,
3,18.03.2018 14:14:50,iuzov.kr17@physics.msu.ru,Юзов К Р,Кирилл,Романович,,,,,113,...,,,,,,,,,,
4,18.03.2018 19:57:49,aandrezv@gmail.com,Зверев А А,Алексей,Андреевич,https://drive.google.com/open?id=1vJ976jQlEZG8...,https://drive.google.com/open?id=172x9QMX18kdj...,,https://drive.google.com/open?id=1He8vTNqKe1C-...,109,...,,,,,,,,,,
5,18.03.2018 20:10:07,korol.a.k99@mail.ru,Король А К,Александр,Кириллович,,,,,105,...,,,,,,,,,,
6,18.03.2018 20:13:46,romanovskii.io17@physics.msu.ru,Романовский Я О,Ярослав,Олегович,https://drive.google.com/open?id=1MfSzxSRLJx3w...,,,,110,...,,,,,,,,,,
7,21.03.2018 0:09:14,anyaerofeeva1999@gmail.com,Ерофеева А В,Анна,Владимировна,https://drive.google.com/open?id=1kR4ZF-DOyC2H...,https://drive.google.com/open?id=1PEQUSzQCUnvR...,https://drive.google.com/open?id=1lxn5GI1p-RjT...,https://drive.google.com/open?id=1SIVA488PiWcY...,113,...,,,,,,,,,,
8,21.03.2018 11:15:26,vashpekazel@gmail.com,Кулёв М С,Михаил,Сергеевич,https://drive.google.com/open?id=1QMMGVVcGBOBg...,https://drive.google.com/open?id=1mfSGMAY9nF1z...,https://drive.google.com/open?id=1zuYlLmvXTgx5...,https://drive.google.com/open?id=1SABP3LrfTwrK...,102,...,,,,,,,,,,
9,21.03.2018 11:26:07,zuev.nv17@physics.msu.ru,Зуев Н В,Никита,Викторович,https://drive.google.com/open?id=1cxBgXXz_57m7...,https://drive.google.com/open?id=1ftLq5oz5kdLi...,,https://drive.google.com/open?id=1VZ-Nz9SVFFT8...,107,...,,,,,,,,,,


In [16]:
# Add node_name of main list (which course and faculty)
NODE_DICT = {'Физический1': "закрытый_список_2017"}#'закр список'}  #, ''}
def find_node_name(key, node_dict):
    if key in node_dict:
        return node_dict[key]
    else:
        return ''
df['node_name'] = [find_node_name(str(a) + str(b), NODE_DICT) 
                   for a, b in zip(df['Факультет'], df['Курс'])]


In [ ]:
df

In [ ]:
wks_copy.set_dataframe(df, start='A1')

In [ ]:
c.spreadsheetId # for sinc add 
#https://docs.google.com/spreadsheets/d/https://docs.google.com/spreadsheets/d/ + id

# Multiple sinc

We have form with multiple faculty and groups. We edit keys and values to main standart at the begining and add column "node dict" to contain name of link at graph sheet. Now we take form as df, sort by "node dict", open link and sync.

In [17]:
def open_node_by_name(node_name, graph_df):
    row = util.get_row_by_keys(graph_df, key_column=["node_name"], key_value=[node_name])
    link = row["node"].values[0]
    return util.read_sheet(util.get_sheet_id_from_link(link))

In [18]:
def get_ind(df, colname, ifval):
    return [str(i[1]) for i in zip(df[colname] == ifval, df.index) if i[0]]

In [19]:
import util
graph_df = util.read_sheet('1MeAYhENafzQMoTKDG-VOLI308ih1wl7SjMLbOkwJU4M').get_as_df()
args = {"to_key_colname":[ "ФИО", "Группа"], "to_values_colname": ["Секция"]}
unique = list(df["node_name"].unique())
if '' in unique:
    unique.remove('')
while unique:
    main_name = unique.pop()
    main_wks = open_node_by_name(main_name, graph_df)
    from_wks = df[df["node_name"] == main_name]
    util.sync_by_colname(from_wks=from_wks, to_wks=main_wks, **args)
    df_sync_ind = get_ind(df, "node_name", main_name)


node_name==закрытый_список_2017
1
ФИО==Волконский Д А&Группа==0
2


TypeError: must be str, not int

In [ ]:
df["mark"] = 0

In [ ]:
import json
from util import read_sheet, get_sheet_id_from_link, sync_by_colname

"""
Go by graph and sync all linked sheets by args.
"""

#graph = read_sheet('1MeAYhENafzQMoTKDG-VOLI308ih1wl7SjMLbOkwJU4M').get_as_df()
graph = read_sheet('1q8z_9QDwSia1IMo7qvDdH2cui0-D5My0xkClopMWcqw').get_as_df() # TEST


for i, line in graph.iterrows():
    if line['Pass']:
        continue

    from_wks = read_sheet(get_sheet_id_from_link(line['node']))
    links = line['link'].split(',')
    args = json.loads(line['args'])
    for link in links:
        to_wks = read_sheet(get_sheet_id_from_link(link))
        print(args['to_values_colname'], line['node'], link)
        sync_by_colname(from_wks=from_wks, to_wks=to_wks, **args)

# 2018 Optimize lessons


In [20]:
import pygsheets
import pandas as pd
import util
import random
import hashlib

In [3]:
import importlib

importlib.reload(util)
importlib.reload(PandasForm)
importlib.reload(SyncGraph)

NameError: name 'PandasForm' is not defined

In [21]:
c = pygsheets.authorize(outh_file='other.json')


#sh = c.open_by_key('1cWN-sMoaV_5NjWTcjryRx-6uxi29K9KaWt5vHa9vQog')#"1-xmNwRbxBwcZSHem67cVhf-ilG_bh_g_MRvR96Q74nE")#
#wks = sh.sheet1


## Read only rules test

In [5]:
def safe_transfer_to_pd_by_col(wks, col_list):
    """
        take col one by one and create pd dataframe (more stable then get_as_df)
        col_list = [1, 2, 6...]
    """ 
    df = pd.DataFrame()
    for col in col_list:
        
        full_col = wks.get_col(col, include_empty=False)
        #print(col, full_col, full_col[0], len(full_col), df.shape[0])
        diff_coll = df.shape[0] - len(full_col) + 1
        if diff_coll > 0: 
            for i in range(diff_coll):
                full_col.append('')
            
        df[full_col[0]] = full_col[1:]
    return df

In [6]:
def score_to_int(sc_val):
    if sc_val == "":
        return 0
    return int(sc_val.split('/')[0])

In [7]:
wks = c.open_by_key(key='1cWN-sMoaV_5NjWTcjryRx-6uxi29K9KaWt5vHa9vQog').sheet1
df = safe_transfer_to_pd_by_col(wks, [1,2,3,4,5,6,7,8]) # 7 problem

In [8]:
# Rename columns from test
col_to_rename = {"Номер учебной группы (101, 201, 201э)":'group', 
                 "Email Address":'key', # emeil hash
                 "Score":'test_rool', # transform to int
                 'Фамилия и инициалы (Пример: Петров А. Б.)':'suname',
                 "Медицинская группа (на загруженной справке)":'med'
                }
df.rename(columns=col_to_rename, inplace=True)

In [9]:
#email -> hash
def mail_hesh(mail):
    return hashlib.md5(mail.encode('UTF-8')).hexdigest()


In [10]:
def get_cource(group_number):
    
    if group_number:
        
        for letter in str(group_number):
            if letter.isdigit():
                return str(letter)           
    return ''
    #return group_number or group_number[0]


In [11]:

df['key'] = df['key'].apply(mail_hesh)
df['key'] = df['key'].apply(str) #int to long for google docs
df['cource'] = df['group'].apply(get_cource)
#df['key'] = df['key'].apply(lambda x: 'a'+str(x))

In [12]:
#test score to int 
df['test_rool'] = df['test_rool'].apply(score_to_int)

In [13]:
df

,Timestamp,key,test_rool,suname,group,Факультет,Фото справки медосмотра,Медицинская группа (на загруженной справке),cource
0,9/4/2018 12:12:35,98937156f7fa66285d0a45ff6b675a51,5,а а,1,Физифак,,основная,1
1,9/21/2018 10:47:21,69a302a1e4948ccd6ceef0969642165d,10,Агапова Д.А.,103,Физифак,https://drive.google.com/open?id=1z9-r983B5H49...,основная,1
2,9/8/2018 13:23:30,ea124424181caae9d3b59b7f3852b33b,24,Альперович К. Д.,209,Физифак,,основная,2
3,9/5/2018 21:05:23,83a0b25fb3f6a9e8b855ba721063a57d,13,Андреев А.А.,212,Физифак,,основная,2
4,9/4/2018 23:01:51,d3ba39bad34c57d41350f16f2e7aef5d,16,Беляев М.Е.,203,Физифак,,основная,2
5,10/10/2018 14:35:21,4b74edf16084b48aaee12091cd184992,20,Бердышев Р.В.,103,Физифак,https://drive.google.com/open?id=1vd0LAAZZZ_7q...,основная,1
6,10/10/2018 0:21:17,3f30ed35239ad67c1fd3209c5a6b6b98,23,Власов В. А.,105,Физифак,https://drive.google.com/open?id=1awIIbAu-_t5Z...,основная,1
7,9/21/2018 23:11:06,6afaa8ad15c0829fe00f7141ff5dfb40,9,Гаврилов В.Р.,105,Физифак,https://drive.google.com/open?id=1JZpGOjhIklCF...,основная,1
8,9/8/2018 13:43:51,bf937329cfa2bc22aefd71dacc11269e,19,Гатина М.И.,212,Физифак,,основная,2
9,9/15/2018 21:42:29,463597944bd1d764b15fc0eb3cbd41c7,19,Давыдов Д.А.,225м,Физифак,,основная,2


## Filter only satisfy results, course and faculty

In [14]:
#get list with same property (as faculty and test_rool > 10 )
#TODO for group filter 201m?
ndf = util.get_row_by_keys(df, key_value=['Физифак', '1'], key_column=['Факультет','cource'])
ndf = util.get_row_by_keys(ndf, ['17',], ['test_rool'], ratio=">")

Факультет==Физифак&cource==1
1
test_rool>17


In [15]:
# add dates to dataframe
col_name=['key', 'suname', 'test_rool', 'group']
dates = util.get_dates(3,3,7)
df2 = pd.DataFrame(
            [[''] * len(dates)], 
            index=ndf.index, 
            columns=dates
        )
new_df = pd.concat([ndf[col_name], df2], axis=1)
new_df.index = range(ndf.shape[0])

In [16]:
##get needed col
#col_name=['key', 'suname', 'test_rool', 'group']
new_df#[col_name]

,key,suname,test_rool,group,10-10,10-14,10-17,10-21,10-24,10-28,...,12-26,12-30,01-02,01-06,01-09,01-13,01-16,01-20,01-23,01-27
0,4b74edf16084b48aaee12091cd184992,Бердышев Р.В.,20,103,,,,,,,...,,,,,,,,,,
1,3f30ed35239ad67c1fd3209c5a6b6b98,Власов В. А.,23,105,,,,,,,...,,,,,,,,,,
2,d468770bd01e255822a94f7f48c0e128,Дубровский,18,106,,,,,,,...,,,,,,,,,,
3,dc60470dabae6e859aa3f16514a595db,Коноплицкий В. В.,20,109,,,,,,,...,,,,,,,,,,
4,078e3c820c94f2f142e433bb161e57df,Короткова Т. А.,23,110,,,,,,,...,,,,,,,,,,
5,789707242dc6dc20add7f60f82352195,Лаптев Р. А.,21,109,,,,,,,...,,,,,,,,,,
6,074039d26b9968ebdeaf80db61c6ad48,Останин Г.С.,19,113,,,,,,,...,,,,,,,,,,
7,d4539871e364d6181cd75d08be7f5bb2,Корнилов Д. В.,20,113,,,,,,,...,,,,,,,,,,
8,db8abdb744c99602dfabf4da8f5dc153,Кузнецов И. А.,18,110,,,,,,,...,,,,,,,,,,
9,1352dc9a015334863ded646ecf500c48,Шутов М. С.,20,118,,,,,,,...,,,,,,,,,,


In [17]:
import PandasForm, SyncGraph


In [18]:
def search_id_in_list(list_sheets, name):
    for sheet in list_sheets:
        if sheet['name'] == name:
            return sheet['id']
    return None

### Change tittle for new sheet

In [19]:
# check add key or create new form
folder_id='1OnF9ZpSiO5IYB4Ns15PZgsTc14DVRr3z'
tittle="ph1"
fc = PandasForm.SheetCreator(title=tittle, folder_id=folder_id)
if fc.check_shet_in_folder(pg=c, sheet_name=fc.title):
    # update with insert key
    sg = SyncGraph.Graph(None)
    wks = c.open_by_key(search_id_in_list(c.list_ssheets(folder_id), tittle))
    sg.add_key(new_df, wks.sheet1, key_col=['key']) #.sheet1
    args = {"to_key_colname": ["key",], 
            "to_values_colname": ['suname', 'test_rool', 'group'],            
           }
    util.sync_by_colname(from_wks=new_df[col_name], to_wks=wks.sheet1, **args)
else:
    #create new
    fc.df = new_df #[col_name]
    fc.create_sheet(pg=c)

In [48]:
#util.get_dates(5,3,7)

# Generate teams

In [28]:
import datetime
todaydate = datetime.datetime.now().strftime("%m-%d")
tomorow = df.columns[df.columns.get_loc(todaydate) + 1]
today_col_ind = df.columns.get_loc(todaydate)

In [25]:
folder_id='1OnF9ZpSiO5IYB4Ns15PZgsTc14DVRr3z'
tittle = 'ch2'
wks = c.open_by_key(search_id_in_list(c.list_ssheets(folder_id), tittle))

In [30]:
players_count = sum([int(s or 0) for s in df[todaydate]])

In [37]:
if players_count % 3 == 0:
    teams_count = players_count // 3 - 1
else:
    teams_count = players_count // 3

In [40]:
team_names = "abcdefghigklmnopq"
players_count / 3
team_list = []
for i in team_names[:teams_count]:
    team_list.extend(list(i * 3))
team_list.extend(list('R' * (players_count % 3 or 3)))

In [41]:
import random
random.shuffle(team_list)

In [43]:
i = 0
for ind, v in enumerate(df[todaydate]):
    if v:
        df[tomorow][ind] = team_list[i]
        i += 1

/home/denis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [44]:
freshdf = wks.sheet1.get_as_df()

In [ ]:
freshdf['today'] = freshdf[todaydate]

In [ ]:
util.get_row_by_keys(freshdf, key_column=['today',], key_value=[1,])

In [ ]:
freshdf

In [48]:
df["10-13"]

0     a
1      
2     b
3      
4     R
5      
6     R
7     b
8     R
9      
10     
11    a
12    a
13     
14     
15     
16    b
17     
Name: 10-13, dtype: object

In [50]:
wks.sheet1.update_col(index=today_col_ind + 2,row_offset=1,values=df[tomorow].values.tolist())

### DOWN WHY NEED&

In [ ]:
#col location
today_col_ind = df.columns.get_loc(todaydate)
df[df.columns[today_col_ind]]

In [ ]:
fc = PandasForm.FormCreator(title="ff2", folder_id='1OnF9ZpSiO5IYB4Ns15PZgsTc14DVRr3z')
fc.df = df[col_name]
fc.create_sheet(pg=c)

In [ ]:
c.list_ssheets(parent_id='1OnF9ZpSiO5IYB4Ns15PZgsTc14DVRr3z')

In [ ]:
sh = c.open('ff2')

In [ ]:
sh.sheet1.get_as_df()

In [ ]:
score_to_int(df["Score"][0]) > 19

In [ ]:
from GDrive import GDrive
g = GDrive(sheet_id='1cWN-sMoaV_5NjWTcjryRx-6uxi29K9KaWt5vHa9vQog',
                   graph='1at8ivfcSDq3W_LFdwzFqVuYfQ27c18Ka-F1Cpen844g'
                   )
#g.generate(category=['Секция', 'Факультет'], keys=['ФИО', 'Группа'],
#                   current_folder='19PZiIxtt9nvLDHst4EZp-FMWYQO4yrKa',
#                   graph_wks=util.read_sheet('1ZR38L8tFMdUTODVQ0cdz79jPKX4xpNn7R11S82u5Jyk'))

# Add Points for Relay

Form sheet add date label. Only one ref can send all winners data for lesson. If date today or early read it and mark as red. Then remove teams from main list and protect it

In [52]:
from TransferFunctions import BasketballResults
bres = BasketballResults('1GUCw3XNS0zWYgjv6dUACc5IoteSYft3DRW58vrDCIKA')

In [7]:
import importlib

importlib.reload(TransferFunctions)

<module 'TransferFunctions' from '/home/denis/Program/graph2/TransferFunctions.py'>

In [53]:
import TransferFunctions

In [54]:
bres = TransferFunctions.BasketballResults('1GUCw3XNS0zWYgjv6dUACc5IoteSYft3DRW58vrDCIKA')
df = bres.get_df()


In [55]:
df

,Timestamp,ref1,ref23,faculty,relay_1,relay_2,relay_3,relay_4,relay_5,relay_6,relay_7,marked
0,9/14/2018 16:11:41,ffbskt@gmail.com,msufizra@gmail.com,Физики2,bce,eba,,,,,,
1,9/14/2018 16:55:35,ffbskt@gmail.com,,Физики2,abc,,,,,,,


In [60]:
# people wineer
list(df["relay_1"][len(df["relay_1"]) - 1])

['a', 'b', 'c']

In [61]:
immrdiatly read abc and +points

SyntaxError: invalid syntax (<ipython-input-61-43a61e649ce4>, line 1)

In [ ]:
for i in df_students:
    ind = df_students.rows.get_loc(i)
    

### down sheet


In [36]:
ref_point = 2
# df 
for row in df.iterrows():
    ans = []
    #print(row[1][11])
    if row[1][11] and int(row[1][11]) < 12:
        print(row[10])
        if int(row[1][11]) == 1: # add ref keys
            ans.extend([(mail_hesh(q), ref_point) for q in row[1][2].split()]) 
            #mark wks 2
        for i in range(min(4, row[1][11]), 11):
            print(row[1][i])
            #ans.extend(#teampoints)
                
    elif:
        ans.append(ref_point)
        ans.extend([(mail_hesh(q), ref_point) for q in row[1][2].split()])
        
        

In [13]:
df.columns[[1]]

Index(['ref1'], dtype='object')

In [21]:
a = ''
[(mail_hesh(q), 2) for q in a.split()]

[]

In [42]:
[1,2] + [2, 7]

[1, 2, 2, 7]

In [ ]:
def add_points_by_keys(keys, edited_df, val_col, points, key_col='key'): #  df[2].split() + df[[0]]
    for key in keys:
        #ad .. apply( + == * points)

In [ ]:
def add points_by_mails(mails, edited_df, val_col, points, key_col='key'):
    keys = [mail_hesh(mail) for mail in mails]
    return add_points_by_keys(keys, edited_df, val_col, points, key_col)

In [ ]:
def add_points_by_teams(team_places_str, team_key_dict): #f(place)
    for place, team in enumerate(team_places_str.split()):
        keys = team_key_dict[team]
        add_points_by_keys(keys, edited_df, val_col, 1 - place * 0.25, key_col)

In [ ]:
fac_to_sheet = {'Физики1':'addid',
                'Физики2':'',
                'Экономисты1':'',
                'Экономисты2':'',
                'Химики1':'addid',
                'Химики2':'1nYVmnNtnTewo8lxBklCTe1GRSzdxKgJYXgXdqiVXW0g'
               }

In [ ]:
def read_teams_result(relay_wks):
    df = relay_wks.get_as_df()
    
    


In [54]:
df['Timestamp'].apply(lambda x: '-'.join(x.split('/')[0:2]))

0    9-14
1    9-14
Name: Timestamp, dtype: object

In [58]:
datetime.datetime(df['Timestamp'])

TypeError: cannot convert the series to <class 'int'>

In [ ]:
self.df = self.safe_load(self.col_list)
        self.df.rename(columns=self.col_to_rename, inplace=True)

# Protected Range

In [54]:
c = pygsheets.authorize(outh_file='other.json')


In [55]:
wks = c.open('ch2').sheet1

In [70]:
wks.create_named_range("a", (1, 10), (5, 15))

AttributeError: 'DataRange' object has no attribute '_protected'

In [65]:
gridrange = {
  "sheetId": 0,
  "startRowIndex": 0,
  "endRowIndex": 2,
  "startColumnIndex": 0,
  "endColumnIndex": 2,
}
pr = wks.create_protected_range(gridrange)


In [ ]:
wks.create_named_range((1, 10), (5, 15))

In [71]:
wks.get_values((1, 10), (5, 15), returnas='range')

AttributeError: 'DataRange' object has no attribute '_protected'

In [ ]:
pr.update()